In [45]:
#https://www.kaggle.com/garethjns/microsoft-lightgbm-with-parameter-tuning-0-823/code

In [79]:
def gera_folds(periodos_treino, folds_cv=5):
    
    folds = []
    
    tamanho_janela = len(periodos_treino) - folds_cv
    assert tamanho_janela > 0, 'Erro! O número de folds deve ser menor'+ \
                             'que o número de meses do treino.'
    
    for i in range(folds_cv):
        
        fold_atual = {}
        fold_atual['treino'] = periodos_treino[i:i+tamanho_janela]
        fold_atual['teste'] = [periodos_treino[tamanho_janela + i]]
        
        print (i, fold_atual)
        
        folds.append(fold_atual)
        
    return folds
        

In [80]:
import itertools

def gera_parametros():
    gridParams = {
            'boosting_type': ['gbdt'],
            'max_depth' : [-1,4,6],
            'objective': ['regression'], 
            'num_leaves': [10,25,50],
            'learning_rate': [0.005, 0.05, 0.5, 1],
            'max_bin': [512], 
            'subsample_for_bin': [200],
            'subsample' : [0.7,1],
            'subsample_freq': [1], 
            #'colsample_bytree' : [0.6, 0.8],
            'reg_alpha' : [1.5,5],
            'reg_lambda' : [5,10],
            'min_split_gain': [0.5], 
            #'min_child_weight': [1], 
            #'min_child_samples': [5], 
            #'scale_pos_weight': [1],
            #'num_class' : [1],
            'n_estimators': [10,25]
        }

    combinacoes = []
    lista_parametros = list(gridParams.keys())

    for valores in itertools.product(*[gridParams[x] for x in lista_parametros]):
        combinacao_atual = {}
        for i in range(len(valores)):
            combinacao_atual[lista_parametros[i]] = valores[i]
        combinacoes.append(combinacao_atual)

    print ('Num combinacoes:', len(combinacoes))
    return combinacoes

In [81]:
# Gera base iris para testes

import pandas as pd
import random


base = pd.read_csv('iris.csv')


meses = []
for i in range(base.shape[0]):
        meses.append(random.randint(1,10))
base['mes'] = meses


variaveis_explicativas = ['var_1','var2','var_3','var_4']
variavel_resposta = 'classe'

In [82]:
# Declaracao de quais periodos serao usados para treinmo e para teste
periodos = {}
periodos['treino'] = [1,3,4,5,6,7,8,9]
periodos['teste'] = [2,10]

# Gera os folds para uma validacao cruzada com janela deslizante
folds = gera_folds(periodos['treino'])
print (folds)

# Gera o vetor com todas as combinacoes de parametros
parametros = gera_parametros()

0 {'treino': [1, 3, 4], 'teste': [5]}
1 {'treino': [3, 4, 5], 'teste': [6]}
2 {'treino': [4, 5, 6], 'teste': [7]}
3 {'treino': [5, 6, 7], 'teste': [8]}
4 {'treino': [6, 7, 8], 'teste': [9]}
[{'treino': [1, 3, 4], 'teste': [5]}, {'treino': [3, 4, 5], 'teste': [6]}, {'treino': [4, 5, 6], 'teste': [7]}, {'treino': [5, 6, 7], 'teste': [8]}, {'treino': [6, 7, 8], 'teste': [9]}]
Num combinacoes: 576


### Encontra os melhores parâmetros para o modelo

In [83]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score


contagem_parametros = 0
resultados = []

melhor_resultado = 0
melhores_parametros = None

# Para cada combinacao de parametro
for parametros_atuais in parametros:
    
    contagem_parametros += 1
    resultados_cv = []
    print ('Processando combinação parâmetros', contagem_parametros)
    
    for i, fold in enumerate(folds):
        
        print ('\tRodando fold', i)
        
        X_treino = base[base['mes'].isin(fold['treino'])][variaveis_explicativas].values
        y_treino = base[base['mes'].isin(fold['treino'])][variavel_resposta].tolist()
        
        X_teste = base[base['mes'].isin(fold['teste'])][variaveis_explicativas].values
        y_teste = base[base['mes'].isin(fold['teste'])][variavel_resposta].tolist()
        
        modelo = lgb.LGBMClassifier(**parametros_atuais)
        modelo.fit(X_treino, y_treino)
        y_pred = modelo.predict(X_teste)
        
        resultados_cv.append(roc_auc_score(y_teste, y_pred))
        
    resultado_atual = resultados_cv.mean()
    
    if resultado_atual > melhor_resultado:
        melhor_resultado = resultado_atual
        melhores_parametros = parametros_atuais
    
    print ('Melhor resultado atual', melhor_resultado)
    
    resultados.append(resultado_atual)
    


Processando combinação parâmetros 1
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 2
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 3
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 4
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 5
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 6
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 7
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando c

Processando combinação parâmetros 67
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 68
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 69
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 70
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 71
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 72
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 73
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Proces

	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 127
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 128
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 129
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 130
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 131
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 132
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 133
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2

Processando combinação parâmetros 190
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 191
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 192
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 193
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 194
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 195
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 196
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0

	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 254
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 255
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 256
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 257
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 258
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 259
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 260

	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 319
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 320
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 321
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 322
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 323
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 324
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 325
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2

	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 385
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 386
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 387
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 388
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 389
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 390
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 391
	Rodando fold 0
	Rodando fold 1

	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 450
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 451
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 452
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 453
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 454
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 455
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 456
	Rodando fold 0

Melhor resultado atual 0
Processando combinação parâmetros 509
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 510
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 511
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 512
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 513
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 514
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4
Melhor resultado atual 0
Processando combinação parâmetros 515
	Rodando fold 0
	Rodando fold 1
	Rodando fold 2
	Rodando fold 3
	Rodando fold 4